<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Numerical methods: 3. Roots
## Multiple equations
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In this notebook, we expand into root-finding with multiple equations and variables.

In [ ]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt

## One function with one variable

$$
\fbox{$f(x)=0$}
$$
with $f(x)=x^2-1$

Roots are $x^0_1=-1$ and $x^0_2=+1$.

Define the $x$ range and the function $f(x)$ as function:

In [ ]:
x=np.linspace(-3,3,21)
def f(x):
    # shifted parabola
    y = x**2 - 1
    return y

In [ ]:
plt.figure(figsize=(8,6))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('f(x)=x$^2$-1')
plt.plot([-3,3],[0,0],linewidth=1,color='grey')
plt.plot(x,f(x))

We use `newton` from the `scipy.optimize` package to find the roots.
1. We first have to **bracket** possible roots ...
2. ..then **search** for the roots.

The bracketing is done in a loop, the calculation of the root with the `newton` method.

**Note:** Despite its name `newton`, the root-finding procedure is the **secant** method,
when we do not hand down a derivative $f'$! Check the manual, e.g. print(scipy.optimize.newton.__doc__)

In [ ]:
x0 = np.array([])
for i in range(len(x)-1):
    if (f(x[i])==0.):
        x0 = np.append(roots,x[i])
    elif (f(x[i+1])==0.):
        x0 = np.append(roots,x[i+1])
    elif (f(x[i])*f(x[i+1])<0):
        root=scipy.optimize.newton(f,(x[i]+x[i+1])/2)
        x0 = np.append(x0,root)
print(x0)

In [ ]:
#print(scipy.optimize.newton.__doc__)

In [ ]:
plt.figure(figsize=(8,6))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('f(x)=x$^2$-1')
plt.plot([-3,3],[0,0],linewidth=1,color='grey')
plt.plot(x,f(x))
plt.plot(x0,np.zeros(len(x0)),linewidth=0,marker='o',markersize=10,color='green')

## One function with two variables

$$
\fbox{$f(x,y)=0$}
$$
with $f(x,y)=xy-1$

Roots are $x^0={{1}\over{y^0}}$.

Define the $x$ and $y$ ranges and the function $f(x,y)$ as function:

In [ ]:
x=np.linspace(-3,3,21)
y=np.linspace(-3,3,21)
X, Y = np.meshgrid(x, y)
def fxy(x,y):
    y= x*y - 1
    return y


In [ ]:
plt.figure(figsize=(8,6))
plt.xlabel('x')
plt.ylabel('y')
plt.title('f(x,y)=xy-1')
plt.contourf(X, Y, fxy(X,Y), 20, cmap='RdGy')
plt.colorbar()
zero=plt.contour(X, Y, fxy(X,Y), 1, colors='black')
plt.clabel(zero, inline=True, fontsize=8)

To find the roots $x^0,y^0$, we need to fix one variable (e.g. $y^0$), then search for the root(s) of the 
other variable (e.g. $x$), and then repeat this for all $y$.

*-not shown-*

## Multiple functions with multiple variables

$$
\fbox{$\vec{F}(\vec{x})=\vec{0}$}
$$

$$
\begin{array}{rcl}
f_1(x_1,x_2) &=& x_1 - x_2 \\
f_2(x_1,x_2) &=& x_1 x_2 -1
\end{array}
$$

Roots are $\vec{x}^0=(1,1)$ or $\vec{x}^0=(-1,-1)$.

We first define the two functions as $f_1$ and $f_2$:

In [ ]:
def f1(x):
    # first function
    f1 = x[0]-x[1]
    return f1

def f2(x):
    # second function
    f2 = x[0]*x[1]-1
    return f2

**Our algorithm**

Initial guess:
$$ 
x_0 = (guess)
$$
Iterative improvement:
$$
x_{i+1} = x_i - J_{ij}^{-1} f_i(x_j), \qquad i=1,n
$$
Jacobi matrix:
$$
 J_{ij}(x_k)  =
 \left[
\begin{array}{cccc}
 \frac{\partial f_1}{\partial x_1}(x_k) &
 \frac{\partial f_1}{\partial x_2}(x_k) &
 \dots &
 \frac{\partial f_1}{\partial x_n}(x_k) \\
 \vdots &&& \vdots \\
 \frac{\partial f_n}{\partial x_1}(x_k) &
 \frac{\partial f_n}{\partial x_2}(x_k) &
 \dots &
 \frac{\partial f_n}{\partial x_n}(x_k) \\
\end{array}
 \right]
$$

We then find the roots by:
1. Arranging the **individual functions** into an array ...
2. Calculate the **Jacobi matrix** of partial derivatives ...
3. **Inverting** the Jacobi matrix

In [ ]:
# start vector
x = [4,+2]
#x = [4,-2]

# iterative improvment
converged=False
while converged != True:
    f    = np.array([f1(x),f2(x)])
    J    = np.array([[1,-1],[x[1],x[0]]])
    invJ = np.linalg.inv(J)
    converged=np.allclose(np.dot(invJ, f), [0,0])
    print (x,converged)
    x[0] = x[0] - invJ[0][0]*f[0] - invJ[0][1]*f[1]
    x[1] = x[1] - invJ[1][0]*f[0] - invJ[1][1]*f[1]

And finally with the `root` function from the `scipy.optimize` package.

We first need to group our two functions into an array of functions, $vecf(x,y)$

In [ ]:
def vecf(x):
    y = [f1(x),f2(x)]   
    return y
print(vecf([4.,2.]))

Then we call the `scipy.optimize.root` method. Check the manual!

It needs the array of function, and array of initial guesses, and the Jacobian matrix.

For the latter, several choices are available... With `jac=False`, we ask scipy to calculate the Jacobian ...

In [ ]:
sol = scipy.optimize.root(vecf, [0, -1], jac=False, method='hybr')
sol.x

Here with an explicit definition of the Jacobian matrix:

In [ ]:
def fun(x):
    return [x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]]

print(fun([0,0]))

def jac(x):
    return np.array([[1 + 1.5 * (x[0] - x[1])**2,
                      -1.5 * (x[0] - x[1])**2],
                     [-1.5 * (x[1] - x[0])**2,
                      1 + 1.5 * (x[1] - x[0])**2]])

print(jac([0,0]))

In [ ]:
sol = scipy.optimize.root(fun, [0, -1], jac=jac, method='hybr')
sol.x

In [ ]:
fun(sol.x)

----
[next >](lib03_roots.ipynb)